# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})
import mlflow
from collections import defaultdict
from scipy.stats import zscore

# original library

import common as com
import pytorch_modeler as modeler
from pytorch_model import DAGMM as Model
from pytorch_utils import filtered_load_model
import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
#with open("./config.yaml", 'rb') as f:
#    config = yaml.load(f)

log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_path = INPUT_ROOT + "/dev_data"
add_dev_path = INPUT_ROOT + "/add_dev_data"
# machine type
MACHINE_TYPE = config['IO_OPTION']['MACHINE_TYPE']
machine_types = os.listdir(dev_path)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
PKL_DIR = OUTPUT_ROOT + '/pkl'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(PKL_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/working/research/dcase2020/result/2D/DAGMM/stable3/covd5e-6/config.yaml'

## make path list and train/valid split

In [7]:
############################################################################
# make path set and train/valid split
############################################################################
'''
train_paths[machine_type]['train' or 'valid'] = path
'''
dev_train_paths = {}
add_train_paths = {}
train_paths = {}

In [8]:
for machine_type in machine_types:
    # dev train
    dev_train_paths = ["{}/{}/train/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(dev_path, machine_type))]
    dev_train_paths = sorted(dev_train_paths)
    # add_dev train
    add_train_paths = ["{}/{}/train/".format(add_dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(add_dev_path, machine_type))]
    add_train_paths = sorted(add_train_paths)
    # valid
    dev_valid_paths = ["{}/{}/test/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/test".format(dev_path, machine_type))]
    dev_valid_paths = sorted(dev_valid_paths)
    
    train_paths[machine_type] = {}
    train_paths[machine_type]['train'] = dev_train_paths + add_train_paths
    train_paths[machine_type]['valid'] = dev_valid_paths

## training

In [9]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(train_paths, machine_type)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)         # debug
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    logger.info('TRAINING')
    # parameter setting
    net = Model(sample_rate=config['preprocessing']['sample_rate'],
                window_size=config['preprocessing']['window_size'],
                hop_size=config['preprocessing']['hop_size'],
                mel_bins=config['preprocessing']['mel_bins'],
                fmin=config['preprocessing']['fmin'],
                fmax=config['preprocessing']['fmax'],
                latent_size=config['fit']['latent_size'],
                mixture_size=config['fit']['mixture_size'])
    #pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_PATH'])
    #net = filtered_load_model(net, pretrained_dict)
    optimizer = optim.Adam(net.parameters(), lr=1e-3)
    num_epochs = config['fit']['num_epochs']
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    criterion = nn.MSELoss()
    #try:
    history = modeler.train_net(net, dataloaders_dict, criterion, optimizer, scheduler, num_epochs, writer)
    #except:
        #history = "error"
        #print("error")
        #return history
    # output
    model = history['model']
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    torch.save(model.state_dict(), model_out_path)
    logger.info('\n success:{0} \n'.format(machine_type) + \
                    'model_out_path ==> \n {0}'.format(model_out_path))
    #  close writer for tensorbord
    writer.close()
    #modeler.mlflow_log(history, config, machine_type, model_out_path, tb_log_dir)
    com.toc()
    return history

In [10]:
machine_types

['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']

In [11]:
for machine_type in machine_types:
    history = run(machine_type)
    #with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    #    pickle.dump(history , file)

2020-11-28 16:46:34,473 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2020-11-28 16:46:34,474 - 00_train.py - INFO - MAKE DATA_LOADER
2020-11-28 16:46:35,501 - 00_train.py - INFO - TRAINING


use: cuda:0


100%|██████████| 29/29 [00:03<00:00,  7.55it/s]
2020-11-28 16:47:19,596 - pytorch_modeler.py - INFO - Epoch 1/300:train_loss:128.568488, tr_rec:36.852177, tr_eng:648.050754, tr_covd:538224.684067, val_AUC:0.513566, val_pAUC:0.510743
100%|██████████| 29/29 [00:03<00:00,  7.67it/s]
2020-11-28 16:47:57,042 - pytorch_modeler.py - INFO - Epoch 2/300:train_loss:99.747752, tr_rec:24.478705, tr_eng:751.249851, tr_covd:2881.206728, val_AUC:0.531515, val_pAUC:0.502570
100%|██████████| 29/29 [00:03<00:00,  7.31it/s]
2020-11-28 16:48:35,676 - pytorch_modeler.py - INFO - Epoch 3/300:train_loss:65.520704, tr_rec:6.079510, tr_eng:593.238750, tr_covd:2346.364775, val_AUC:0.512117, val_pAUC:0.505538
100%|██████████| 29/29 [00:04<00:00,  7.23it/s]
2020-11-28 16:49:13,351 - pytorch_modeler.py - INFO - Epoch 4/300:train_loss:20.093910, tr_rec:2.283094, tr_eng:177.011863, tr_covd:2192.597582, val_AUC:0.509639, val_pAUC:0.501566
100%|██████████| 29/29 [00:03<00:00,  7.30it/s]
2020-11-28 16:49:51,324 - pytor

100%|██████████| 29/29 [00:04<00:00,  7.12it/s]
2020-11-28 17:05:45,593 - pytorch_modeler.py - INFO - Epoch 30/300:train_loss:-81.260527, tr_rec:0.058222, tr_eng:-814.581189, tr_covd:2787.437843, val_AUC:0.510159, val_pAUC:0.503686
100%|██████████| 29/29 [00:04<00:00,  7.17it/s]
2020-11-28 17:06:23,585 - pytorch_modeler.py - INFO - Epoch 31/300:train_loss:-82.163217, tr_rec:0.125080, tr_eng:-824.385451, tr_covd:3004.981749, val_AUC:0.489896, val_pAUC:0.496051
100%|██████████| 29/29 [00:03<00:00,  9.20it/s]
2020-11-28 17:07:01,039 - pytorch_modeler.py - INFO - Epoch 32/300:train_loss:-63.797786, tr_rec:0.066239, tr_eng:-640.101350, tr_covd:2922.218149, val_AUC:0.479957, val_pAUC:0.509050
100%|██████████| 29/29 [00:02<00:00, 10.96it/s]
2020-11-28 17:07:27,861 - pytorch_modeler.py - INFO - Epoch 33/300:train_loss:-63.827723, tr_rec:0.055844, tr_eng:-641.837906, tr_covd:6004.510832, val_AUC:0.523043, val_pAUC:0.499864
100%|██████████| 29/29 [00:02<00:00, 10.06it/s]
2020-11-28 17:07:54,845 

100%|██████████| 29/29 [00:02<00:00, 10.86it/s]
2020-11-28 17:17:44,212 - pytorch_modeler.py - INFO - Epoch 59/300:train_loss:-102.716845, tr_rec:0.118862, tr_eng:-1032.060634, tr_covd:7407.143188, val_AUC:0.518070, val_pAUC:0.511643
100%|██████████| 29/29 [00:02<00:00, 10.94it/s]
2020-11-28 17:18:10,657 - pytorch_modeler.py - INFO - Epoch 60/300:train_loss:-113.804707, tr_rec:0.079180, tr_eng:-1141.676795, tr_covd:5675.888945, val_AUC:0.517827, val_pAUC:0.512234
100%|██████████| 29/29 [00:02<00:00, 10.51it/s]
2020-11-28 17:18:37,006 - pytorch_modeler.py - INFO - Epoch 61/300:train_loss:-111.931934, tr_rec:0.055509, tr_eng:-1134.382607, tr_covd:29016.388542, val_AUC:0.543726, val_pAUC:0.511873
100%|██████████| 29/29 [00:02<00:00, 10.80it/s]
2020-11-28 17:19:03,838 - pytorch_modeler.py - INFO - Epoch 62/300:train_loss:-107.096889, tr_rec:0.053132, tr_eng:-1076.384403, tr_covd:9768.427389, val_AUC:0.534186, val_pAUC:0.506082
100%|██████████| 29/29 [00:02<00:00, 10.62it/s]
2020-11-28 17:1

100%|██████████| 29/29 [00:02<00:00, 11.11it/s]
2020-11-28 17:30:32,118 - pytorch_modeler.py - INFO - Epoch 88/300:train_loss:-82.790908, tr_rec:0.057464, tr_eng:-830.240114, tr_covd:3512.840082, val_AUC:0.500182, val_pAUC:0.500010
100%|██████████| 29/29 [00:02<00:00, 10.97it/s]
2020-11-28 17:30:58,296 - pytorch_modeler.py - INFO - Epoch 89/300:train_loss:-102.389615, tr_rec:0.032945, tr_eng:-1026.095810, tr_covd:3740.453176, val_AUC:0.500000, val_pAUC:0.500000
100%|██████████| 29/29 [00:02<00:00, 11.04it/s]
2020-11-28 17:31:24,108 - pytorch_modeler.py - INFO - Epoch 90/300:train_loss:-103.676088, tr_rec:0.030278, tr_eng:-1038.283149, tr_covd:2439.016096, val_AUC:0.540587, val_pAUC:0.501712
100%|██████████| 29/29 [00:02<00:00, 10.33it/s]
2020-11-28 17:31:50,426 - pytorch_modeler.py - INFO - Epoch 91/300:train_loss:-100.861396, tr_rec:0.025320, tr_eng:-1009.957818, tr_covd:2181.366796, val_AUC:0.559556, val_pAUC:0.503611
100%|██████████| 29/29 [00:02<00:00, 10.94it/s]
2020-11-28 17:32:1

100%|██████████| 29/29 [00:02<00:00, 11.94it/s]
2020-11-28 17:43:17,696 - pytorch_modeler.py - INFO - Epoch 117/300:train_loss:-119.094221, tr_rec:0.009287, tr_eng:-1194.288012, tr_covd:6505.902020, val_AUC:0.500058, val_pAUC:0.500001
100%|██████████| 29/29 [00:02<00:00, 11.75it/s]
2020-11-28 17:43:44,089 - pytorch_modeler.py - INFO - Epoch 118/300:train_loss:-119.130478, tr_rec:0.007682, tr_eng:-1241.648066, tr_covd:100532.980522, val_AUC:0.500000, val_pAUC:0.500000
100%|██████████| 29/29 [00:02<00:00, 11.46it/s]
2020-11-28 17:44:10,274 - pytorch_modeler.py - INFO - Epoch 119/300:train_loss:-131.084708, tr_rec:0.005172, tr_eng:-1315.964823, tr_covd:10132.067587, val_AUC:0.524764, val_pAUC:0.490790
100%|██████████| 29/29 [00:02<00:00, 11.79it/s]
2020-11-28 17:44:36,691 - pytorch_modeler.py - INFO - Epoch 120/300:train_loss:-134.895034, tr_rec:0.005205, tr_eng:-1350.885198, tr_covd:3765.707050, val_AUC:0.476296, val_pAUC:0.505008
100%|██████████| 29/29 [00:02<00:00, 11.43it/s]
2020-11-2

100%|██████████| 29/29 [00:02<00:00, 11.08it/s]
2020-11-28 17:55:59,416 - pytorch_modeler.py - INFO - Epoch 146/300:train_loss:-82.325518, tr_rec:0.025088, tr_eng:-826.192530, tr_covd:5372.972826, val_AUC:0.457465, val_pAUC:0.491269
100%|██████████| 29/29 [00:02<00:00, 11.02it/s]
2020-11-28 17:56:25,449 - pytorch_modeler.py - INFO - Epoch 147/300:train_loss:-92.083470, tr_rec:0.013761, tr_eng:-927.691125, tr_covd:13437.660061, val_AUC:0.500000, val_pAUC:0.500000
100%|██████████| 29/29 [00:02<00:00, 10.38it/s]
2020-11-28 17:56:51,990 - pytorch_modeler.py - INFO - Epoch 148/300:train_loss:-109.400454, tr_rec:0.007020, tr_eng:-1110.598133, tr_covd:33046.842232, val_AUC:0.520293, val_pAUC:0.496727
100%|██████████| 29/29 [00:02<00:00, 11.17it/s]
2020-11-28 17:57:18,568 - pytorch_modeler.py - INFO - Epoch 149/300:train_loss:-106.744236, tr_rec:0.006176, tr_eng:-1071.055538, tr_covd:7102.895353, val_AUC:0.478798, val_pAUC:0.497938
100%|██████████| 29/29 [00:02<00:00, 10.99it/s]
2020-11-28 17:

KeyboardInterrupt: 

history = run('ToyCar')
with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    pickle.dump(history , file)